In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py 
from f2py_jit import jit
from tqdm import tqdm
from config_io_module import read_cnf_atoms
from joblib import delayed, Parallel

main_dir = './fortran_main/'
gcm = jit(main_dir+'gcm_module.f90')
mc = jit([main_dir+'seed_module.f90',main_dir+'gcm_module.f90',main_dir+'montecarlo_module.f90'], flags='-O3 -ffast-math')

gr_dir = '/mnt/c/Users/cucch/Documents/4.UNI/LAB_SIMULAZIONI_AM/lab2/hardspheres/'
gr = jit(gr_dir+'gr_module.f90')

path = '/mnt/c/Users/cucch/Documents/4.UNI/LAB_SIMULAZIONI_AM/lab1/hardspheres'
init = jit([os.path.join(path, 'config_io_module.f90'),
           os.path.join(path, 'maths_module.f90'),
           os.path.join(path, 'initialize_module.f90'),
           os.path.join(path, 'init_mymodule2.f90')], flags='-O3 -ffast-math')

def visualize_3dmol(positions, cell=None, colors=None, radii=None,
                    center=None, color='white', radius=0.5, gas=False):
    """
    Visualize a particle configuration using 3dmol http://3dmol.csb.pitt.edu/
    """
    import py3Dmol
    if center is None:
        center = [0.5] * len(cell)
    if colors is None:
        colors = [color] * len(positions)
    if radii is None:
        radii = [radius] * len(positions)
    view = py3Dmol.view()
    view.setBackgroundColor('white')
    for i in range(len(positions)):
        view.addSphere({'center': {'x': positions[i][0],
                                   'y': positions[i][1],
                                   'z': positions[i][2]},
                        'radius': radii[i], 'color': colors[i]})
    if gas==False: 
        for i in range(1, len(positions)):
            view.addCylinder({'start': {'x': positions[i-1][0],
                                    'y': positions[i-1][1],
                                    'z': positions[i-1][2]},
                            'end': {'x': positions[i][0],
                                    'y': positions[i][1],
                                    'z': positions[i][2]},
                            'color': 'white'})
    if cell is not None:
        view.addBox({'center': {'x': center[0], 'y': center[1], 'z': center[2]},
                     'dimensions': {'w': cell[0], 'h': cell[1], 'd': cell[2]},
                     'wireframe': True, 'color': "#000000"})
    return view

In [2]:
class System: 
    
    def __init__(self,filename,seed):
        self.filename = filename  # contains the informations about the system
        self.seed = seed
        self.N = 0 
        self.box_lenght = 0 
        self.rho = 0 
        self.box = [] 
        self.position = [] 
        self.eo = 0
        self.wo = 0 
    
    def initialize(self):
        self.N, self.box_lenght, pos = read_cnf_atoms(f'./initial/{self.filename}')
        self.position = pos.transpose()
        self.box = np.array([self.box_lenght, self.box_lenght, self.box_lenght])
        self.rho = self.N/(self.box_lenght)**3
        self.eo, self.wo = mc.montecarlo.interaction(self.position,self.box)
        
        
class Simulation: 
    
    def __init__(self,particle_system,nsteps,dr):
        self.system = particle_system   # connection to the system class
        self.nsteps = nsteps            # total number of steps
        self.dr = dr
        self.count = np.array([0])      # accepted configuration count
        self.position = self.system.position
        self.position_list = []
        
    def simulation(self,T): 
        mc.montecarlo.put_seed(self.system.seed)
        for i in tqdm(range(self.nsteps)):
            mc.montecarlo.mc_step(self.position,self.count,self.dr,self.system.box,T)
            if i%10 == 0: 
                temporary_position = np.copy(self.position)
                self.position_list.append(temporary_position)
        
class Analysis: 
    
    def __init__(self,particle_system,simulation,nbin,dr):
        self.system = particle_system 
        self.sim = simulation 
        self.nbin = nbin
        self.max_distance = dr
        self.e = [] 
        self.w = [] 
        self.bins = []
        self.ghisto = []
        
    def get_energy_virial(self): 
        for i in range(len(self.sim.position_list)):
            pos = self.sim.position_list[i]
            ep, wp = mc.montecarlo.interaction(pos,self.system.box)
            self.e.append(ep) ; self.w.append(wp)
        return np.array(self.e), np.array(self.w)
    
    def compute_ew(self,position):
        e,w = mc.montecarlo.interaction(position,box=self.system.box)
        return e,w
    
    def get_ew(self):
        data = Parallel(n_jobs=-1)(delayed(self.compute_ew)(self.sim.position_list[i]) for i in range(len(self.sim.position_list)))
        e = np.array(data)[:,0]
        w = np.array(data)[:,1]
        return e, w 
    
    def get_gr(self,nequil):
        histo_list = []
        for i in range(len(self.sim.position_list)):
            pos = self.sim.position_list[i]
            bins = np.zeros(self.nbin) ; histo = np.zeros(self.nbin,dtype=int)
            gr.distribution_function.gr_self(pos,self.system.box,self.max_distance,histo,bins)
            omega =2.0*np.pi*self.max_distance*(bins**2)*self.system.rho*self.system.N
            histo_list.append(histo/omega)
        self.bins = bins
        self.ghisto = np.mean(histo_list[nequil:],axis=0)
        return self.bins, self.ghisto
    
    def compute_rdf(self,position):
        bins = np.zeros(self.nbin) ; histo = np.zeros(self.nbin,dtype=int)
        gr.distribution_function.gr_self(position,self.system.box,self.max_distance,histo,bins)
        omega =4*np.pi*self.max_distance*bins**2
        return bins, histo/omega 
    
    def get_rdf(self):
        data = Parallel(n_jobs=-1)(delayed(self.compute_rdf)(self.sim.position_list[i]) for i in range(len(self.sim.position_list)))
        bins = np.array(data)[0][0]
        hist = np.array([np.array(data[i][1]) for i in range(np.array(data).shape[0])])
        return bins, hist
    
class Heating: 
    
    def __init__(self,particle_system,simulation,data_collection,T_list):
        self.system = particle_system
        self.sim = simulation
        self.analysis = data_collection 
        self.T_range = T_list
        self.pos = []
        self.e_mean = np.zeros(len(self.T_range))
        self.w_mean = np.zeros(len(self.T_range))
        self.bins = []
        self.gr_mean = []
        
    def temperature_scan(self):
        for i in range(len(self.T_range)):
            t = self.T_range[i]
            self.pos = self.system.position
            self.sim.count = np.array([0]) 
            self.sim.position_list = []
            self.sim.simulation(T=t)
            en, wn = self.analysis.get_energy_virial()
            # en, wn = self.analysis.get_ew()
            self.e_mean[i] = np.mean(en[3000:]) ; self.w_mean[i] = np.mean(wn[3000:])
            # self.bins, hist = self.analysis.get_rdf()
            # self.gr_mean.append(np.mean(hist,axis=0))
            print('Temperature : {} Acceptance ratio : {}'.format(t,self.sim.count[0]/(self.system.N*self.sim.nsteps)))
            
def savetxt(filename,position_array): 
    with h5py.File(filename, 'w') as f:
        f.create_dataset('positions', data=position_array)
            

##### Isoterma

In [5]:
path = '/rho_list/'
tfolder = 'T_015_new'
t = 0.0015
os.makedirs(f'./{tfolder}') 

for i in range(20):
    # load the initial configuration
    conf = System(path+f'r_{i+1}.inp',seed=85)
    conf.initialize()
    # decide the number of steps and start the simulation
    if conf.rho >= 0.6 : 
        steps = 30000
    else : 
        steps = 50000 
    sim = Simulation(particle_system=conf,nsteps=steps,dr=0.1)
    sim.simulation(T=t)
    # store the configuration data in a h5 file
    savetxt(f'{tfolder}/pos_{i+1}.h5',sim.position_list)

100%|██████████| 50000/50000 [03:23<00:00, 245.73it/s]
